---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import itertools
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        filename, _ = urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified %s' % filename)
    else:
        print(statinfo.st_size)
        raise Exception(
          'Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
    with zipfile.ZipFile(filename) as f:
        name = f.namelist()[0]
        data = tf.compat.as_str(f.read(name))
    return data
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


In [4]:
letters = sorted(set((string.ascii_letters + ' ').lower()))
translate_table = {ord(l): ord(l) for l in letters}

In [5]:
bigrams = {c1 + c2: i for i, (c1, c2) in 
           enumerate(itertools.product(letters, letters))}
inversed_bigrams = {v: k for k, v in bigrams.items()}

vocabulary_size = len(bigrams)

In [6]:
def id_2_bigram(x):
    return inversed_bigrams[x]

def text_2_bigrams(text):
    text = text.translate(translate_table)
    if len(text) % 2 != 0:
        text += ' '
    return np.array([bigrams[text[i:i + 2]] for i in range(0, len(text), 2)])

In [7]:
class BatchGenerator(object):
    def __init__(self, text, batch_size, num_unrollings):
        self._text = text
        self._text_size = len(text)
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        segment = self._text_size // batch_size
        self._cursor = [offset * segment for offset in range(batch_size)]
        self._last_batch = self._next_batch()

    def _next_batch(self):
        """Generate a single batch from the current cursor position in the data."""
        batch = np.zeros(
            shape=(self._batch_size), dtype=np.int16)
        for b in range(self._batch_size):
            batch[b] = self._text[self._cursor[b]]
            self._cursor[b] = (self._cursor[b] + 1) % self._text_size
        return batch

    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by num_unrollings new ones.
        """
        batches = [self._last_batch]
        for step in range(self._num_unrollings):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches
    
class BigramBatchGenerator(object):
    def __init__(self, text, batch_size, num_unrollings=0):
        bigrams = text_2_bigrams(text)
        self._generator = BatchGenerator(bigrams, batch_size, num_unrollings)

    def next(self):
        return self._generator.next()
    
def characters(bigram_ids):
    return [id_2_bigram(b_id) for b_id in bigram_ids]

def batches2string(batches):
    """Convert a sequence of batches back into their (most likely) string
    representation."""
    s = [''] * batches[0].shape[0]
    for b in batches:
        s = [''.join(x) for x in zip(s, characters(b))]
    return s

In [8]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


In [9]:
batch_size = 64
num_unrollings = 10

train_batches = BigramBatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BigramBatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

['ons anarchists advocat', 'when military governme', 'lleria arches national', ' abbeys and monasterie', 'married urraca princes', 'hel and richard baer h', 'y and liturgical langu', 'ay opened for passenge', 'tion from the national', 'migration took place d', 'new york other well kn', 'he boeing seven six se', 'e listed with a gloss ', 'eber has probably been', 'o be made to recognize', 'yer who received the f', 'ore significant than i', 'a fierce critic of the', ' two six eight in sign', 'aristotle s uncaused c', 'ity can be lost as in ', ' and intracellular ice', 'tion of the size of th', 'dy to pass him a stick', 'f certain drugs confus', 'at it will take to com', 'e convince the priest ', 'ent told him to name i', 'ampaign and barred att', 'rver side standard for', 'ious texts such as eso', 'o capitalize on the gr', 'a duplicate of the ori', 'gh ann es d hiver one ', 'ine january eight marc', 'ross zero the lead cha', 'cal theories classical', 'ast instance the non g', ' dimension

In [10]:
num_nodes = 64
embedding_size = 256

graph = tf.Graph()
with graph.as_default():
    keep_prob = tf.placeholder(tf.float32)
    embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1, 1))
    
    IW = tf.Variable(tf.truncated_normal([embedding_size, num_nodes * 4], -0.1, 0.1))
    OW = tf.Variable(tf.truncated_normal([num_nodes, num_nodes * 4], -0.1, 0.1))
    B = tf.Variable(tf.zeros([1, num_nodes * 4]))
    
    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))
    
    def lstm_cell(i, o, state):
        gate = tf.matmul(i, IW) + tf.matmul(o, OW) + B
        input_gate = tf.sigmoid(gate[:, 0:num_nodes])
        forget_gate = tf.sigmoid(gate[:, num_nodes:2 * num_nodes])
        update = tf.tanh(gate[:, 2 * num_nodes:3 * num_nodes])
        output_gate = tf.sigmoid(gate[:, 3 * num_nodes:])
        state = forget_gate * state + input_gate * update
        return output_gate * tf.tanh(state), state
    
    train_data = []
    embedded_train_data = []
    
    for _ in range(num_unrollings + 1):
        batch = tf.placeholder(tf.int32, shape=[batch_size])
        train_data.append(batch)
        embedded_batch = tf.nn.embedding_lookup(embeddings, batch)
        embedded_train_data.append(embedded_batch)
        
    train_inputs = embedded_train_data[:num_unrollings]
    train_labels = train_data[1:]

    outputs = []
    output = saved_output
    state = saved_state
    
    for i in train_inputs:
        i = tf.nn.dropout(i, keep_prob)
        output, state = lstm_cell(i, output, state)
        output = tf.nn.dropout(output, keep_prob)
        outputs.append(output)
        
    
    with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
        logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
        labels = tf.one_hot(tf.concat(train_labels, 0), vocabulary_size)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,
                                                                      labels=labels))
    
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)
    
    train_prediction = tf.nn.softmax(logits)
    
    sample_input = tf.placeholder(tf.int32, shape=[1])
    embedded_sample_input = tf.nn.embedding_lookup(embeddings, sample_input)
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    
    reset_sample_state = tf.group(
        saved_sample_output.assign(tf.zeros([1, num_nodes])),
        saved_sample_state.assign(tf.zeros([1, num_nodes]))
    )
    
    sample_output, sample_state = lstm_cell(embedded_sample_input,
                                            saved_sample_output,
                                            saved_sample_state)
    
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                  saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(tf.concat(sample_output, 0), w, b))

In [11]:
def sample(distribution):
    """Sample one element from a distribution assumed
    to be an array of normalized probabilities.
    """
    r = random.uniform(0, 1)
    s = 0
    for i in range(len(distribution)):
        s += distribution[i]
        if s >= r:
            return i
    return len(distribution) - 1

def logprob(predictions, labels):
    """Log-probability of the true labels in a predicted batch."""
    predictions[predictions < 1e-10] = 1e-10
    return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

In [12]:
def one_hot(x, n_classes=vocabulary_size):
    X = np.zeros((len(x), n_classes))
    X[np.arange(len(x)), x] = 1
    return X

In [13]:
train_stat = []

num_steps = 20001
summary_frequency = 200
keep_prob_ = 0.7

# gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
# config = tf.ConfigProto(gpu_options=gpu_options, log_device_placement=True)
config = None

with tf.Session(graph=graph, config=config) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    mean_loss = 0
    
    for step in range(num_steps):
        batches = train_batches.next()
        feed_dict = {train_data_i: batch.reshape((batch_size,)) 
                     for train_data_i, batch in zip(train_data, batches)}
        feed_dict[keep_prob] = keep_prob_
    
        _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate], 
                                            feed_dict=feed_dict)
        mean_loss += l

        if step % summary_frequency == 0:
            if step > 0:
                mean_loss /= summary_frequency
            # The mean loss is an estimate of the loss over the last few batches.
            print('Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
            mean_loss = 0
            labels = np.concatenate(list(batches)[1:])
            train_pp = float(np.exp(logprob(predictions, one_hot(labels))))
            print('Minibatch perplexity: %.2f' % train_pp)
    
            if step % (summary_frequency * 10) == 0:
                # Generate some samples.
                print('=' * 80)

                for _ in range(5):
                    # sample text
                    bigram_id = np.random.randint(0, vocabulary_size) # initial bigram id
                    sample_len = 40


                    text = id_2_bigram(bigram_id)

                    for _ in range(sample_len // 2):
                        prediction = sample_prediction.eval({sample_input: [bigram_id], keep_prob: 1.0})
                        bigram_id = sample(prediction[0])
                        text += id_2_bigram(bigram_id)

                    print(text)
                print('=' * 80)
            # Measure validation set perplexity.
            reset_sample_state.run()
            valid_logprob = 0
            for _ in range(valid_size):
                b = valid_batches.next()
                predictions = sample_prediction.eval({sample_input: b[0], keep_prob: 1.0})
                valid_logprob += logprob(predictions, one_hot(b[1]))
                
            valid_pp = float(np.exp(valid_logprob / valid_size))
            print('Validation set perplexity: %.2f' % valid_pp)
            
            train_stat.append({
                'step': step,
                'train_pp': train_pp,
                'valid_pp': valid_pp
            })

Initialized
Average loss at step 0: 6.601061 learning rate: 10.000000
Minibatch perplexity: 735.88
axwongbgwmsefteiqnbjumywamhtkcdigllx g uhm
pjbl sr gnipuzdbbuspkabqmyydxkzrrbiggqumey
pqhcuf icgepyhrksdpqordickhzzhyveggzjmgimn
ryfrdxgnoivavnvaefykltobapywqqyddsnvkpkiwu
 ywfciottkmumewol nwuiton exearxxpmu rkmsi
Validation set perplexity: 658.60
Average loss at step 200: 4.610123 learning rate: 10.000000
Minibatch perplexity: 62.37
Validation set perplexity: 54.67
Average loss at step 400: 3.982599 learning rate: 10.000000
Minibatch perplexity: 49.98
Validation set perplexity: 42.36
Average loss at step 600: 3.871839 learning rate: 10.000000
Minibatch perplexity: 53.76
Validation set perplexity: 33.81
Average loss at step 800: 3.801738 learning rate: 10.000000
Minibatch perplexity: 43.68
Validation set perplexity: 33.15
Average loss at step 1000: 3.713014 learning rate: 10.000000
Minibatch perplexity: 43.28
Validation set perplexity: 30.54
Average loss at step 1200: 3.716199 learning r

Validation set perplexity: 21.65
Average loss at step 10200: 3.550837 learning rate: 0.100000
Minibatch perplexity: 37.38
Validation set perplexity: 21.63
Average loss at step 10400: 3.540404 learning rate: 0.100000
Minibatch perplexity: 29.73
Validation set perplexity: 21.58
Average loss at step 10600: 3.488560 learning rate: 0.100000
Minibatch perplexity: 38.01
Validation set perplexity: 21.55
Average loss at step 10800: 3.511946 learning rate: 0.100000
Minibatch perplexity: 39.60
Validation set perplexity: 21.50
Average loss at step 11000: 3.502890 learning rate: 0.100000
Minibatch perplexity: 26.98
Validation set perplexity: 21.47
Average loss at step 11200: 3.516047 learning rate: 0.100000
Minibatch perplexity: 36.30
Validation set perplexity: 21.39
Average loss at step 11400: 3.510704 learning rate: 0.100000
Minibatch perplexity: 34.48
Validation set perplexity: 21.30
Average loss at step 11600: 3.532881 learning rate: 0.100000
Minibatch perplexity: 44.94
Validation set perplexit